In [87]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [88]:
# Libraries for Data Visualisation
import os
import cv2
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
from matplotlib.image import imread
from sklearn.metrics import fbeta_score
from tqdm import tqdm
import tensorflow as tf
from keras import optimizers
from tensorflow.keras.models import Sequential 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input , Dense , Dropout , Flatten,\
Conv2D,MaxPooling2D , BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History
import tensorflow_addons as tfa
import warnings
warnings.filterwarnings('ignore')


In [89]:
path = "../input/planets-dataset/"
os.listdir(path)

['planet', 'test-jpg-additional']

In [90]:
train_df = pd.read_csv('/kaggle/input/planets-dataset/planet/planet/train_classes.csv')
train_df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [91]:
train_df['tags'].nunique()

449

In [92]:
#Getting Unique tags, using property that ssets only contain unique values

def unique_tags(tags):
    labels = set()
    for tag_list in tags:
        for tag in tag_list.split():
            labels.add(tag)
    
    return list(labels)

train_df1 = train_df.copy()
unique_tags = unique_tags(train_df1['tags'])
print(unique_tags)

print('\n','No unique tags:', len(unique_tags))



['blow_down', 'water', 'habitation', 'blooming', 'partly_cloudy', 'bare_ground', 'selective_logging', 'slash_burn', 'clear', 'cloudy', 'conventional_mine', 'artisinal_mine', 'primary', 'road', 'cultivation', 'haze', 'agriculture']

 No unique tags: 17


In [93]:
# One hot encoding is performed on the labels in the train dataset
# The lambda function checks if the current tag is present in the list of tags obtained from the 'tags' column
# If the tag is present, assign the value 1 to the corresponding column in traindf1
# Otherwise, assign 0
for tag in unique_tags:
    train_df1[tag] = train_df1['tags'].apply(lambda x: 1 if tag in x.split() else 0)
    

# Adding .jpg extension to the 'image_name' column
train_df1['image_name'] = train_df1['image_name'].apply(lambda x: f"{x}.jpg")
# The lambda function appends the '.jpg' extension to the image names in the 'image_name' column of the train dataset

train_df1.head()


,image_name,tags,blow_down,water,habitation,blooming,partly_cloudy,bare_ground,selective_logging,slash_burn,clear,cloudy,conventional_mine,artisinal_mine,primary,road,cultivation,haze,agriculture
0,train_0.jpg,haze primary,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1,train_1.jpg,agriculture clear primary water,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1
2,train_2.jpg,clear primary,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,train_3.jpg,clear primary,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,train_4.jpg,agriculture clear habitation primary road,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1


In [94]:
#importing libraries for training
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [95]:
# Defining the columns,i.e the labels that were newly added to the train_classes via hot encoding.
columns = train_df1.columns[2:].tolist()
len(columns)

17

In [96]:
columns

['blow_down',
 'water',
 'habitation',
 'blooming',
 'partly_cloudy',
 'bare_ground',
 'selective_logging',
 'slash_burn',
 'clear',
 'cloudy',
 'conventional_mine',
 'artisinal_mine',
 'primary',
 'road',
 'cultivation',
 'haze',
 'agriculture']

In [97]:
def fbeta(y_true, y_pred, beta = 2, epsilon = 1e-4):
    '''
    Set y_true and y_pred

    Args:
        y_true: correct target values
        Y_pred: predicted values returned by the classifer
        beta = 2
        epsilon= 1e-4
        
    Returns:
        fbeta score
    '''
    
    beta_squared = beta**2
    
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.greater(tf.cast(y_pred, tf.float32), tf.constant(0.5)), tf.float32)
    
    tp = tf.reduce_sum(y_true * y_pred, axis = 1)
    fp = tf.reduce_sum(y_pred, axis = 1) - tp
    fn = tf.reduce_sum(y_true, axis = 1) - tp
    
    precision = tp/(tp+fp+epsilon)
    recall = tp/(tp+fn+epsilon)
    
    fb = (1+beta_squared)*precision*recall / (beta_squared*precision+recall+epsilon)
    return fb

In [98]:
def multi_label_acc(y_true, y_pred, epsilon = 1e-4):
    '''
    Retuns accuracy value for multi_label classification
    
    Set y_true and y_pred

    Args:
        y_true: correct target values
        Y_pred: predicted values returned by the classifer
        epsilon= 1e-4
        
    Returns:
        Accuracy score
    '''
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.greater(tf.cast(y_pred, tf.float32), tf.constant(0.5)), tf.float32)
    
    tp = tf.reduce_sum(y_true * y_pred, axis = 1)
    fp = tf.reduce_sum(y_pred, axis = 1) - tp
    fn = tf.reduce_sum(y_true, axis = 1) - tp
    
    y_true = tf.cast(y_true, tf.bool)
    y_pred = tf.cast(y_pred, tf.bool)
        
    tn = tf.reduce_sum(tf.cast(tf.logical_not(y_true), tf.float32)
                       * tf.cast(tf.logical_not(y_pred), tf.float32), axis = 1)
    
    return (tp+tn)/(tp+tn+fp+fn+epsilon)

In [99]:
#defining our model
def build_model():
    model = Sequential()
    model.add(BatchNormalization(input_shape=(128, 128, 3)))
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(17, activation='sigmoid'))

    opt = Adam(lr=1e-4)
    
    # We need binary here, since categorical_crossentropy l1 norms the output before calculating loss.
    model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=[multi_label_acc, fbeta])

    return model

In [100]:
#modelcheckpoint is set to monitor the model using validation fbeta score and save the best only
save_best_check_point = ModelCheckpoint(filepath = 'best_model.hdf5', 
                                        monitor = 'val_fbeta',
                                        mode = 'max',
                                        save_best_only = True,
                                        save_weights_only = True)

In [101]:
#initializing imagedatagenerator with a validation split of 0.2
train_image_gen = ImageDataGenerator(rescale = 1/255, validation_split = 0.2)

#generating train data generator which is 80% of the train dataset
#note that a generator contains both features and target of the data
train_generator = train_image_gen.flow_from_dataframe(dataframe=train_df1,
                                                directory ="/kaggle/input/planets-dataset/planet/planet/train-jpg",  
                                                x_col="image_name", y_col=columns, subset="training", 
                                                batch_size=16,seed=2021, shuffle=True, 
                                                class_mode="raw", target_size=(128,128))

#generating validation data which is expected to be 20% of the train dataset since validation split is 0.2
val_generator = train_image_gen.flow_from_dataframe(dataframe=train_df1,
                                                directory ="/kaggle/input/planets-dataset/planet/planet/train-jpg",  
                                                x_col="image_name", y_col=columns, subset="validation", 
                                                batch_size=16,seed=2021, shuffle=True, 
                                                class_mode="raw", target_size=(128,128))

Found 32384 validated image filenames.
Found 8095 validated image filenames.


In [102]:
import numpy as np
#setting up step size for training and validation image data
step_train_size = int(np.ceil(train_generator.samples / train_generator.batch_size))
step_val_size = int(np.ceil(val_generator.samples / val_generator.batch_size))

In [103]:
#initialize the model
model1 = build_model()

In [104]:
# Preview the model architecture
model1.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_1 (Batc  (None, 128, 128, 3)      12        
 hNormalization)                                                 
                                                                 
 conv2d_20 (Conv2D)          (None, 128, 128, 32)      896       
                                                                 
 conv2d_21 (Conv2D)          (None, 126, 126, 32)      9248      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 63, 63, 32)       0         
 g2D)                                                            
                                                                 
 dropout_15 (Dropout)        (None, 63, 63, 32)        0         
                                                                 
 conv2d_22 (Conv2D)          (None, 63, 63, 64)       

In [105]:
#fitting our model using the parameters already defined 
model1.fit(x = train_generator, 
           steps_per_epoch = step_train_size, 
           validation_data = val_generator, 
           validation_steps = step_val_size,epochs = 10, 
           callbacks=[save_best_check_point])

Epoch 1/10


2023-07-18 07:13:26.777263: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_5/dropout_15/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


2024/2024 [==============================] - 99s 46ms/step - loss: 0.2339 - multi_label_acc: 0.9096 - fbeta: 0.6530 - val_loss: 0.2120 - val_multi_label_acc: 0.9161 - val_fbeta: 0.6772
Epoch 2/10
2024/2024 [==============================] - 92s 45ms/step - loss: 0.2471 - multi_label_acc: 0.9076 - fbeta: 0.6507 - val_loss: 0.2557 - val_multi_label_acc: 0.9050 - val_fbeta: 0.6424
Epoch 3/10
2024/2024 [==============================] - 90s 45ms/step - loss: 0.2569 - multi_label_acc: 0.9052 - fbeta: 0.6430 - val_loss: 0.2559 - val_multi_label_acc: 0.9050 - val_fbeta: 0.6424
Epoch 4/10
2024/2024 [==============================] - 93s 46ms/step - loss: 0.2569 - multi_label_acc: 0.9052 - fbeta: 0.6430 - val_loss: 0.2560 - val_multi_label_acc: 0.9050 - val_fbeta: 0.6424
Epoch 5/10
2024/2024 [==============================] - 93s 46ms/step - loss: 0.2567 - multi_label_acc: 0.9052 - fbeta: 0.6430 - val_loss: 0.2557 - val_multi_label_acc: 0.9050 - val_fbeta: 0.6424
Epoch 6/10
2024/2024 [=========

In [106]:
#initializing a second model to make predictions
model2 = build_model()

In [107]:
#loading in the weights of the trained model
model2.load_weights('best_model.hdf5')

In [108]:
##adding .jpg extension to image name in the sample submission file
sample_submission = pd.read_csv('/kaggle/input/planets-dataset/planet/planet/sample_submission.csv')
sample_submission1 = sample_submission.copy()
sample_submission1['image_name'] = sample_submission1['image_name'].apply(lambda x: '{}.jpg'.format(x))
sample_submission1.head()

,image_name,tags
0,test_0.jpg,primary clear agriculture road water
1,test_1.jpg,primary clear agriculture road water
2,test_2.jpg,primary clear agriculture road water
3,test_3.jpg,primary clear agriculture road water
4,test_4.jpg,primary clear agriculture road water


In [109]:
# Divide the sample submission file into two splits,
# first test_df which contains the first 40669 images 
test_df = sample_submission1.iloc[:40669]['image_name'].reset_index().drop('index', axis =1)
test_df.head()

,image_name
0,test_0.jpg
1,test_1.jpg
2,test_2.jpg
3,test_3.jpg
4,test_4.jpg


In [110]:
#initialize imagedatagenerator for the test images and also rescaling
test_image_gen = ImageDataGenerator(rescale = 1/255)

#creating a generator for the images found in the first test image files
test_generator = test_image_gen.flow_from_dataframe(dataframe=test_df, 
                                                directory="/kaggle/input/planets-dataset/planet/planet/test-jpg", 
                                                x_col="image_name", 
                                                y_col=None, 
                                                batch_size=16, 
                                                shuffle=False, 
                                                class_mode=None, 
                                                target_size=(128,128))

step_test_size = int(np.ceil(test_generator.samples/test_generator.batch_size))



Found 40669 validated image filenames.


In [111]:
#first, we reset the test generator to avoid shuffling of index as we want it to be orderly
test_generator.reset()
pred = model2.predict(test_generator, steps = step_test_size, verbose = 1)

2542/2542 [==============================] - 82s 32ms/step


In [113]:
#this is to get the filenames in the generator using the attribute .filenames
file_names = test_generator.filenames

#convert the predicted values to a dataframe and join two labels together if the probability of occurrance 
#of the label is greater than 0.5 
pred_tags = pd.DataFrame(pred)
pred_tags = pred_tags.apply(lambda x: ' '.join(np.array(unique_tags)[x>0.5]), axis = 1)

#then the result should look like this 
result1 = pd.DataFrame({'image_name': file_names, 'tags': pred_tags})
result1.head()

,image_name,tags
0,test_0.jpg,clear primary
1,test_1.jpg,clear primary
2,test_2.jpg,primary
3,test_3.jpg,clear primary
4,test_4.jpg,primary


In [114]:
#additional test dataset
add_test_df = sample_submission1.iloc[40669:]['image_name'].reset_index().drop('index', axis =1)
add_test_df.head()

,image_name
0,file_0.jpg
1,file_1.jpg
2,file_10.jpg
3,file_100.jpg
4,file_1000.jpg


In [115]:
#creating a generator for the additional test image files
add_test_generator = test_image_gen.flow_from_dataframe(dataframe = add_test_df, 
                                                    directory ="/kaggle/input/planets-dataset/test-jpg-additional/test-jpg-additional", 
                                                    x_col="image_name", 
                                                    y_col=None, 
                                                    batch_size=16, 
                                                    shuffle=False, 
                                                    class_mode=None, 
                                                    target_size=(128,128))


step_test_size2 = int(np.ceil(add_test_generator.samples/add_test_generator.batch_size))

Found 20522 validated image filenames.


In [116]:
#we reset the generator to avoid shuffling, then make prediction on the generator
add_test_generator.reset()
add_pred = model2.predict(add_test_generator, steps = step_test_size2, verbose = 1)

1283/1283 [==============================] - 123s 96ms/step


In [120]:
#this is to get the filenames in the generator using the attribute .filenames
file_names = add_test_generator.filenames

#convert the predicted values to a dataframe and join two labels together if the probability of occurrance 
#of the label is greater than 0.5
add_pred_tags = pd.DataFrame(add_pred)
add_pred_tags = add_pred_tags.apply(lambda x: ''.join(np.array(unique_tags)[x>0.5]), axis = 1)

#then the result should look like this
result2 = pd.DataFrame({'image_name': file_names, 'tags': add_pred_tags})
result2.head()

,image_name,tags
0,file_0.jpg,clearprimary
1,file_1.jpg,clearprimary
2,file_10.jpg,primary
3,file_100.jpg,clearprimary
4,file_1000.jpg,clearprimary


In [121]:
#for the final result of the predicted tags for the test images,
# we need to concat the first and second results in 
#that order to avoid shuffling the index
last_result = pd.concat([result1, result2])

last_result = last_result.reset_index().drop('index', axis =1)

print(last_result.shape)
#print the final result
last_result.head()

(61191, 2)


,image_name,tags
0,test_0.jpg,clear primary
1,test_1.jpg,clear primary
2,test_2.jpg,primary
3,test_3.jpg,clear primary
4,test_4.jpg,primary


In [122]:
#we need to remove the .jpg extension from the image_name of the
# last_result because from the sample submission the 
#extension was not there, we added it for easy manipulation of the data.
last_result['image_name'] = last_result['image_name'].apply(lambda x: x[:-4])
last_result.head()

,image_name,tags
0,test_0,clear primary
1,test_1,clear primary
2,test_2,primary
3,test_3,clear primary
4,test_4,primary


In [123]:
# Finally, we save the result to a csv file using the .to_csv() 
# method and setting the index to false.
last_result.to_csv('submission1.csv', index = False)